# 1 pacotes 

In [1]:
import mwclient
import time
import datetime as dt
from datetime import date, timedelta
import pandas as pd
import json

## 2 Obter MatchId a partir da data 

In [27]:
def obterMatchId(dataJogo:str,**kwargs):
    
    liga = kwargs.get('liga','')
    dataJogoFim = kwargs.get('dataJogoFim','')
    
    date = dt.datetime.strptime(dataJogo, "%Y-%m-%d").date()
    dateFinal = dt.datetime.strptime(dataJogoFim, "%Y-%m-%d").date() if len(dataJogoFim)>0 else date
    
    cond = "M.DateTime_UTC >= '" + str(date) + " 00:00:00' AND M.DateTime_UTC <= '" + str(dateFinal+dt.timedelta(1)) + " 00:00:00'"
    if len(liga)>0:
        cond = cond+" AND "+f"M.MatchId like '%{liga}%'"
    
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = 'max',
        tables = "MatchSchedule=M",
        fields = "M.MatchId,M.DateTime_UTC,M.PageAndTeam1, M.PageAndTeam2, M.Team1,M.Team2, M.BestOf,M.Patch,M.PatchPage,M.VodHighlights",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [29]:
obterMatchId('2022-07-02', liga = 'CBLOL',dataJogoFim='2022-07-03')

,MatchId,DateTime UTC,PageAndTeam1,PageAndTeam2,Team1,Team2,BestOf,Patch,PatchPage,VodHighlights,DateTime UTC__precision
0,CBLOL/2022 Season/Split 2_Week 4_1,2022-07-02 16:00:00,CBLOL/2022 Season/Split 2_Miners,CBLOL/2022 Season/Split 2_Flamengo Los Grandes,Miners,Flamengo Los Grandes,1,12.12,Patch 12.12,,0
1,CBLOL/2022 Season/Split 2_Week 4_10,2022-07-03 20:00:00,CBLOL/2022 Season/Split 2_Flamengo Los Grandes,CBLOL/2022 Season/Split 2_FURIA Esports,Flamengo Los Grandes,FURIA Esports,1,12.12,Patch 12.12,,0
2,CBLOL/2022 Season/Split 2_Week 4_2,2022-07-02 17:00:00,CBLOL/2022 Season/Split 2_Rensga Esports,CBLOL/2022 Season/Split 2_LOUD,Rensga Esports,LOUD,1,12.12,Patch 12.12,,0
3,CBLOL/2022 Season/Split 2_Week 4_3,2022-07-02 18:00:00,CBLOL/2022 Season/Split 2_RED Canids,CBLOL/2022 Season/Split 2_KaBuM! e-Sports,RED Canids,KaBuM! e-Sports,1,12.12,Patch 12.12,,0
4,CBLOL/2022 Season/Split 2_Week 4_4,2022-07-02 19:00:00,CBLOL/2022 Season/Split 2_paiN Gaming,CBLOL/2022 Season/Split 2_FURIA Esports,paiN Gaming,FURIA Esports,1,12.12,Patch 12.12,,0
5,CBLOL/2022 Season/Split 2_Week 4_5,2022-07-02 20:00:00,CBLOL/2022 Season/Split 2_INTZ,CBLOL/2022 Season/Split 2_Liberty,INTZ,Liberty,1,12.12,Patch 12.12,,0
6,CBLOL/2022 Season/Split 2_Week 4_6,2022-07-03 16:00:00,CBLOL/2022 Season/Split 2_KaBuM! e-Sports,CBLOL/2022 Season/Split 2_Rensga Esports,KaBuM! e-Sports,Rensga Esports,1,12.12,Patch 12.12,,0
7,CBLOL/2022 Season/Split 2_Week 4_7,2022-07-03 17:00:00,CBLOL/2022 Season/Split 2_Liberty,CBLOL/2022 Season/Split 2_RED Canids,Liberty,RED Canids,1,12.12,Patch 12.12,,0
8,CBLOL/2022 Season/Split 2_Week 4_8,2022-07-03 18:00:00,CBLOL/2022 Season/Split 2_INTZ,CBLOL/2022 Season/Split 2_paiN Gaming,INTZ,paiN Gaming,1,12.12,Patch 12.12,,0
9,CBLOL/2022 Season/Split 2_Week 4_9,2022-07-03 19:00:00,CBLOL/2022 Season/Split 2_LOUD,CBLOL/2022 Season/Split 2_Miners,LOUD,Miners,1,12.12,Patch 12.12,,0


## Obter video de gravação através do MatchId

In [58]:
def obterVideoFromGamedId(macthid):
    
    cond = f"MG.MatchId = '{macthid}'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "MatchScheduleGame=MG",
        fields = "MG.GameId,MG.MatchId,MG.Blue,MG.Red,MG.Vod,MG.VodPostgame,MG.MatchHistory,MG.N_GameInMatch",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [59]:
obterVideoFromGamedId('CBLOL/2022 Season/Split 2_Week 4_1')

,GameId,MatchId,Blue,Red,Vod,VodPostgame,MatchHistory,N GameInMatch
0,CBLOL/2022 Season/Split 2_Week 4_1_1,CBLOL/2022 Season/Split 2_Week 4_1,Miners,Flamengo Los Grandes,https://www.youtube.com/watch?v=oGwlB1RDfYQ,,,1


## Obter informações do time através do GameId

In [70]:
def obterScoreFromGameId(gameId):
    
    cond = f"SG.GameId like '%{gameId}%'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "ScoreboardGames=SG",
        fields = "SG.GameId,RiotGameId,Tournament,SG.DateTime_UTC,SG.Gamelength,Gamelength_Number,Patch,PatchSort,VOD,Team1Dragons,Team1Barons,Team1Towers,Team1Gold,Team1Kills,Team1RiftHeralds,Team1Inhibitors,MatchHistory,Team2Dragons,Team2Barons,Team2Towers,Team2Gold,Team2Kills,Team2RiftHeralds,Team2Inhibitors",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [71]:
obterScoreFromGameId('CBLOL/2022 Season/Split 2_Week 4_1_1')

,GameId,RiotGameId,Tournament,DateTime UTC,Gamelength,Gamelength Number,Patch,PatchSort,VOD,Team1Dragons,...,Team1Inhibitors,MatchHistory,Team2Dragons,Team2Barons,Team2Towers,Team2Gold,Team2Kills,Team2RiftHeralds,Team2Inhibitors,DateTime UTC__precision
0,CBLOL/2022 Season/Split 2_Week 4_1_1,,CBLOL 2022 Split 2,2022-07-02 16:19:00,34:49,34.816667,12.12,,https://www.youtube.com/watch?v=oGwlB1RDfYQ,4,...,1,,1,0,1,55525,6,0,0,0


## Obter informação dos times pelo GameId

In [83]:
def obterTeamScoreFromGameId(gameId,**kwargs):
    
    cond = f"ST.GameId like '%{gameId}%'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "ScoreboardTeams=ST",
        fields = "ST.GameId,ST.Team,IsWinner,Side,Number,GameTeamId,UniqueTeam",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [85]:
ts = obterTeamScoreFromGameId('CBLOL/2022 Season/Split 2_Week 4_1_1')

## Obter informação dos jogadoes pelo GameTeamId

In [97]:
def obterPlayerScoreFromGameTeamId(GameTeamId,**kwargs):
    
    cond = f"SP.GameTeamId like '%{GameTeamId}%'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "ScoreboardPlayers=SP",
        fields = "GameTeamId,Name,Role,Role_Number,IngameRole,Link,Champion,Kills,Deaths,Assists,Gold,CS,DamageToChampions",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [98]:
ts.iloc[0,5]

'CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1_1_Team2'

In [ ]:
obterPlayerScoreFromGameTeamId(ts.iloc[0,5])